## Running core metrics diversity preparation commands in qiime2 on the total sum scaled data

need to first convert my tsv file to a qza file to run the core diversity analysis

In [1]:
## converting my tsv to a biom table as an intermediate
! biom convert \
    -i ../data/qiime/total_sum_scaling.tsv \
    -o ../data/qiime/total_sum_scaling.biom \
    --table-type "Table" \
    --to-hdf5

In [2]:
## converting my new biom table to a qza file 
! qiime tools import \
    --input-path ../data/qiime/total_sum_scaling.biom \
    --type 'FeatureTable[Frequency]' \
    --output-path ../data/qiime/total_sum_scaling.qza

Imported updated/total_sum_scaling3.biom as BIOMV210DirFmt to updated/total_sum_scaling3.qza


In [3]:
! qiime feature-table summarize \
    --i-table ../data/qiime/total_sum_scaling.qza \
    --o-visualization ../data/qiime/total_sum_scaling.qzv

Saved Visualization to: /Users/madiapgar/gut_microbiome_metabolomics/total_sum_scaled/total_sum_scaling2.qzv


converting the rep-seqs to a fasta file (updated as of May 16, 2023)

In [6]:
! qiime tools export \
    --input-path ../data/misc/euk_filt-mergedDietAim1rep-seqs_051523-Copy1.qza \
    --output-path ../data/misc

Exported /Users/madiapgar/gut_microbiome_metabolomics/CaseyandMadi/euk_filt-mergedDietAim1rep-seqs_051523-Copy1.qza as DNASequencesDirectoryFormat to directory /Users/madiapgar/gut_microbiome_metabolomics/total_sum_scaled/updated


need to use my rep seqs file to generate a phylogenetic tree via sepp

In [8]:
! qiime fragment-insertion sepp \
--i-representative-sequences ../data/misc/euk_filt-mergedDietAim1rep-seqs_051523-Copy1.qza \
--o-tree ../data/qiime/tree.qza \
--o-placements ../data/qiime/placements.qza \
## need to fix the next like with the correct file path to the silva database
--i-reference-database inputs/sepp-refs-silva-128.qza

^C


need to filter the sepp phylogenic tree via my biom table to only include relevant ASVs

In [ ]:
! qiime fragment-insertion filter-features \
--i-table ../data/qiime/total_sum_scaling.qza \
--i-tree ../data/qiime/tree.qza \
--o-filtered-table ../data/qiime/total_sum_filt_table.qza \
--o-removed-table ../data/qiime/total_sum_rem_table.qza

generation of taxonomic classification and filtering steps before alpha and beta diversity analysis

In [ ]:
! qiime feature-classifier classify-sklearn \
## same with this next line!!
--i-classifier /Users/madiapgar/Desktop/qiimework/silva-138-99-515-806-nb-classifier.qza \
--i-reads ../data/misc/euk_filt-mergedDietAim1rep-seqs_051523-Copy1.qza \
--o-classification ../data/qiime/taxonomy.qza

In [ ]:
! qiime metadata tabulate \
--m-input-file ../data/qiime/taxonomy.qza \
--o-visualization ../data/qiime/taxonomy.qzv

In [ ]:
## filtering my taxonomic table 
! qiime taxa filter-table \
--i-table ../data/qiime/total_sum_scaling.qza \
--i-taxonomy ../data/qiime/taxonomy.qza \
--p-include p_ \
--p-exclude mitochondria,chloroplast \
--o-filtered-table ../data/qiime/taxonomy_filtered.qza

In [ ]:
## i'm creating a visualization for my taxonomic filtered table and I will be able to use that table for my alpha/beta
## diversity aanlysis and building my taxa bar chart 
! qiime feature-table summarize \
--i-table ../data/qiime/taxonomy_filtered.qza \
--o-visualization ../data/qiime/taxonomy_filtered.qzv

some of my samples couldn't go through core metrics so I had to filter them out of the taxonomic table using my metadata file before I could re-run core metrics


In [ ]:
! qiime feature-table filter-samples \
    --i-table ../data/qiime/taxonomy_filtered.qza \
    --m-metadata-file ../data/misc/merged_metadata1.tsv \
    --o-filtered-table ../data/qiime/tax_filt_actual.qza